In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from xs_lexer import Lexer
from xs_parser import Parser
# from xs_node import Node_letter, Node_condition

In [2]:
class Node_letter(object):

    def __init__(self, token, name):
        self.token = token
        self.name = name
        self.state = 0
        self.neg = 0
        self.childs_pos = []
        self.childs_neg = []
        self.visited = 0
        self.or_dic = {}
        self.xor_dic = {}
        self.ind = 0

    def __str__(self):
        return "Node_letter({})".format(self.token)


class Node_condition(object):

    def __init__(self, left, cond, right):
        self.left = left
        self.token = self.cond = cond
        self.right = right
        self.neg = 0
        self.visited = 0
        self.or_dic = {}
        self.xor_dic = {}
        self.ind = 0

    def __str__(self):
        return "Node_condition({}".format(self.token)

In [191]:
class Interpreter(object):

    def __init__(self):
        pass

    def interpret(self, node, graph):
        if node:
            if type(node).__name__ == "Node_condition":
                return abs(self.apply_logical(node, node.left,
                           node.right, graph) - node.neg)
            node.visited = 1
            if type(node).__name__ == "Node_letter":
                implies_node = graph.find_letter_in_implies(node.name)
                if implies_node:
                    for child in implies_node.childs_pos:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                if child in implies_node.xor_dic.keys():
                                    self.get_ind_fact(graph, implies_node, child)
                                node.state = 1
                    for child in implies_node.childs_neg:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                node.state = 0
                node.visited = 0
                return abs(node.state - node.neg)

    def get_ind_fact(self, graph, implies_node, ind_rule):
        # 1 : copy du graph
        # 2 : reset du graph 
        # 3 : changement des regles
        # 4 : test avec une 3eme fonction similaire a interpret
        # refaire 2-3-4 avec l'autre cas
        
        #1
        copy_graph_1 = copy.deepcopy(graph)
        copy_graph_2 = copy.deepcopy(graph)
        
        # CASE 1
        # get copy_node, equivalent of implies_node in copy_graph
        copy_node = copy_graph_1.find_letter_in_implies(implies_node.name)
        
        # get copy_rule, equivalent of implies_node in copy_graph
        copy_rule = self.get_copy_rule(implies_node, ind_rule, copy_node, copy_graph_1)
        
        
        # del rule in mirror_node_pos, then add it in mirror_node_neg
#         print(copy_node.childs_pos[0])
        del copy_node.xor_dic[copy_rule].childs_pos[0]
        copy_node.xor_dic[copy_rule].childs_neg.append(copy_rule)

        # del copy_rule in xor_dic in mirror_node, then in node
#         print(copy_node.xor_dic[copy_rule].xor_dic[copy_rule])
        del copy_node.xor_dic[copy_rule].xor_dic[copy_rule]
        del copy_node.xor_dic[copy_rule]
        # test
        print(self.interpret_2(copy_node, copy_graph_1))

       

    def interpret_2(self, node, graph):
        if node:
            if type(node).__name__ == "Node_condition":
                return abs(self.apply_logical(node, node.left,
                           node.right, graph) - node.neg)
            node.visited = 1
            if type(node).__name__ == "Node_letter":
                implies_node = graph.find_letter_in_implies(node.name)
                if implies_node:
                    for child in implies_node.childs_pos:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                if child in implies_node.xor_dic.keys():
                                    self.get_ind_fact(graph, implies_node, child)
                                node.state = 1
                    for child in implies_node.childs_neg:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                node.state = 0
                node.visited = 0
                return abs(node.state - node.neg)
            
    def get_copy_rule(self, true_node, true_rule, copy_node, copy_graph):
         # to do
        return copy_node.childs_pos[0]
        
        
    def reset_visit(self, node):
        if node:
            node.visited = 0
            if type(node).__name__ == "Node_letter":
                for child in (node.childs_pos + node.childs_neg):
                    self.reset_visit(child)
            if type(node).__name__ == "Node_condition":
                self.reset_visit(node.left)
                self.reset_visit(node.right)
            
    def apply_logical(self, node, left, right, graph):
        if node.token.type == "AND":
            return self.interpret(left, graph) & self.interpret(right, graph)
        if node.token.type == "OR":
            return self.interpret(left, graph) | self.interpret(right, graph)
        if node.token.type == "XOR":
            return self.interpret(left, graph) ^ self.interpret(right, graph)

In [192]:
# rule1 = "A => (C ^ B) + E"
# rule2 = "F => D ^ (B ^ C)"
rule1 = "A => B ^ C"
rule2 = "B => D"

true_facts = "A"

queries = "D"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
    graph.add_new_AST(Parser(Lexer(rule2)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    graph.check_contradiction()
    
    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

1
D is True


In [27]:
class Graph(object):

    def __init__(self):
        self.implies_list = []
        self.implies_name_list = []
        self.true_facts = ""
        self.interpretor = Interpreter()

    def error(self, s):
        raise Exception(s)

    def add_new_AST(self, root):
        self.update_implies_list(root.right)
        self.update_graph(root.left, root.right)

    def update_implies_list(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.name not in self.implies_name_list:
                    self.implies_list.append(node)
                    self.implies_name_list.append(node.name)
            if type(node).__name__ == "Node_condition":
                self.update_implies_list(node.left)
                self.update_implies_list(node.right)

    def update_graph(self, left, node):
        if node:
#             if node.token.type == "OR" or node.token.type == "XOR":
#                 left.ind = 1
            if type(node).__name__ == "Node_letter":
                if node.neg == 1:
                    (self.find_letter_in_implies(node.name)
                        .childs_neg.append(left))
                else:
                    (self.find_letter_in_implies(node.name)
                        .childs_pos.append(left))
            if type(node).__name__ == "Node_condition":
                if node.token.type == "OR" or node.token.type == "XOR":
                    self.update_ind_dics(left, node)
                self.update_graph(left, node.left)
                self.update_graph(left, node.right)

    def update_ind_dics(self, left, node):
        if node.token.type == "OR":
            if type(node.left).__name__ == "Node_letter":
                self.find_letter_in_implies(node.left.name).or_dic[left] = node.right
            else:
                node.left.or_dic[left] = node.right
            if type(node.right).__name__ == "Node_letter":
                self.find_letter_in_implies(node.right.name).or_dic[left] = node.left
            else:
                node.right.or_dic[left] = node.left
        if node.token.type == "XOR":
            if type(node.left).__name__ == "Node_letter":
                self.find_letter_in_implies(node.left.name).xor_dic[left] = node.right
            else:
                node.left.xor_dic[left] = node.right
            if type(node.right).__name__ == "Node_letter":
                self.find_letter_in_implies(node.right.name).xor_dic[left] = node.left
            else:
                node.right.xor_dic[left] = node.left
            
    def find_letter_in_implies(self, name):
        for node in self.implies_list:
            if node.name == name:
                return node
        return None

    def learn_facts(self, true_facts):
        self.true_facts = true_facts

    def set_facts(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                for child in (node.childs_pos + node.childs_neg):
                    self.set_facts(child)
                if node.token.value in self.true_facts:
                    node.state = 1 
            if type(node).__name__ == "Node_condition":
                self.set_facts(node.left)
                self.set_facts(node.right)

    def query(self, letter):
        if letter in self.full_history.keys():
            print("{} is {}".format(letter, self.full_history[letter]))
            return self.full_history[letter]
        elif letter in self.true_facts:
            print("{} is True".format(letter))
            return True
        else:
            print("{} is False".format(letter))
            return False
        
    def check_contradiction(self):
        self.full_history = {}
        for node in self.implies_list:
            self.full_history[node.name] = self.get_final_state(node)
            if self.full_history[node.name] is None:
                if node.name in self.true_facts:
                    self.full_history[node.name] = True
                else:
                    self.full_history[node.name] = False

    def get_final_state(self, node):
        history = []
        ind_trig = 0
        for child in node.childs_pos:
            res = self.interpretor.interpret(child, self)
#             print(res, node, child)
            if res == 1:
#                 if child in node.xor_dic.keys():
#                     true_state = self.get_undetermined_state()
#                     history.append(true_state)
#                 else:
                    history.append(True)

        for child in node.childs_neg:
            res = self.interpretor.interpret(child, self)
            if res == 1:
                history.append(False)

        if len(set(history)) == 0:
            return
        elif len(set(history)) == 1:
            return history[0]
        else:
            self.error("Error contradiction found with letter {}"
                       .format(node.name))

In [28]:
# rule1 = "A => (C ^ B) + E"
# rule2 = "F => D ^ (B ^ C)"
rule1 = "A => B ^ C"
rule2 = "B => D"

true_facts = "A"

queries = "D"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
    graph.add_new_AST(Parser(Lexer(rule2)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    graph.check_contradiction()
    
    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

D is True


In [16]:
for node in graph.implies_list:
    print(node)
print()
for node in graph.implies_list[0].childs_pos:
    if node in graph.implies_list[0].xor_dic:
        print("ici", node)
    else:
        print("pas ici", node)
print()
# print(graph.implies_list[2].childs_pos)
print(graph.implies_list[0].xor_dic.keys())

Node_letter(Token(LETTER, C))
Node_letter(Token(LETTER, B))
Node_letter(Token(LETTER, E))
Node_letter(Token(LETTER, D))

ici Node_letter(Token(LETTER, A))
ici Node_letter(Token(LETTER, F))

dict_keys([<xs_node.Node_letter object at 0x1041b43c8>, <xs_node.Node_letter object at 0x1041b4e10>])


In [45]:
import copy

In [47]:
print(graph)
print(graph.implies_list)

t = copy.deepcopy(graph)
print(t)
print(t.implies_list)

[<xs_node.Node_letter object at 0x1041b4e10>, <xs_node.Node_letter object at 0x1041b49e8>, <xs_node.Node_letter object at 0x1041b4c18>]
[<xs_node.Node_letter object at 0x1041ce278>, <xs_node.Node_letter object at 0x1041b4240>, <xs_node.Node_letter object at 0x1041b42b0>]


In [21]:
t = copy.copy(graph.implies_list[0])

# for x in t:
print(x)
print()
for x in graph.implies_list[0].childs_pos:
    print(x)


Node_letter(Token(LETTER, F))

Node_letter(Token(LETTER, A))
Node_letter(Token(LETTER, F))


In [37]:
dic = {}
dic['a'] = "b"
dic['c'] = "d"

In [41]:
for i in dic.values():
    print(i)

b
d


In [35]:
dic.keys

<function dict.keys>